In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import argparse 
import helper
import math

import pandas as pd
import numpy as np
import pickle
import tensorflow as tf

import sys
code_dir = './'
sys.path.append(code_dir)
import hughes_data 
import hughes_castnet_model

2023-09-19 14:32:43.208721: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-19 14:32:43.212026: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-19 14:32:43.299360: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-19 14:32:43.300623: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 14:32:52.587393: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
class Config:
    hidden_layer_size = 32
    no_epochs = 300
    num_train_years = 7
    num_test_years = 2
    num_valid_years = 1
    window_size = None
    lead_time = None
    time_unit = 1
    group_lasso = False
    dist = None
    gl_reg_coef = None
    rnn_dropout = 0.1
    temporal_feature_size = None
    static_feature_size = None
    no_locations = None
    batch_size = 50
    learning_rate = 0.005
    embedding_size = 32
    test_time = False
    dataset_name = None
    num_spatial_heads = None
    orthogonal_loss_coef = None

In [4]:
cook_county = False
castnet_datadir = './Data/MA/'

In [5]:
conf = Config()

parser = argparse.ArgumentParser(description='Non-Event or Event')

parser.add_argument('--ip', default='None', type=str)
parser.add_argument('--stdin', default='None', type=str)
parser.add_argument('--control', default='None', type=str)

parser.add_argument('--dataset_name', default='None', type=str)
parser.add_argument('--hidden_size', default=32, type=int)
parser.add_argument('--window_size', default=3, type=int)
parser.add_argument('--lead_time', default=1, type=int)
parser.add_argument('--gl_reg_coef', default=0.0025, type=float)
parser.add_argument('--dropout', default=0.1, type=float)
parser.add_argument('--group_lasso', default='True', type=str)
parser.add_argument('--dist', default='root_d', type=str)
parser.add_argument('--test_time', default='True', type=str)
parser.add_argument('--num_spatial_heads', default=4, type=int)
parser.add_argument('--orthogonal_loss_coef', default=0.01, type=float)

args, _ = parser.parse_known_args()

conf.dataset_name = "Chicago" if cook_county else "MA"
conf.hidden_size = args.hidden_size
conf.window_size = args.window_size
conf.lead_time = args.lead_time
conf.gl_reg_coef = args.gl_reg_coef
conf.dropout = args.dropout
conf.dist = args.dist
conf.num_spatial_heads = args.num_spatial_heads
conf.orthogonal_loss_coef = args.orthogonal_loss_coef
conf.group_lasso = helper.str2bool(args.group_lasso)
conf.test_time = helper.str2bool(args.test_time)

In [6]:
(train_svi_local, train_svi_global, train_static,
    train_sample_indices, train_dist, train_y,
    valid_svi_local, valid_svi_global, valid_static,
        valid_sample_indices, valid_dist, valid_y,
        test_svi_local, test_svi_global, test_static,
            test_sample_indices, test_dist, test_y) = hughes_data.readData(castnet_datadir, conf.dataset_name, conf.window_size, conf.lead_time,
                                                                            conf.num_train_years, conf.num_test_years, conf.num_valid_years,
                                                                            conf.dist, conf.time_unit)

num_years 13
Trimmed svi shape(1620, 13, 5)
Start indicies 0, 7, 8
num_train_years: 7
window_size: 3
lead_time: 1
Difference (num_train_years - (window_size + lead_time - 1)): 4


In [7]:
print(f'train years {train_svi_local.shape[0]/1620}')
print(f'valid years {valid_svi_local.shape[0]/1620}')
print(f'test years {test_svi_local.shape[0]/1620}')

train years 7.0
valid years 1.0
test years 2.0


In [8]:
if(conf.test_time == True):
    print("It's test time, concatting validation data to train")
    train_svi_global = np.concatenate([train_svi_global, valid_svi_global], axis=0)
    train_svi_local = np.concatenate([train_svi_local, valid_svi_local], axis=0)
    train_static = np.concatenate([train_static, valid_static], axis=0)
    train_sample_indices = np.concatenate([train_sample_indices, valid_sample_indices], axis=0)
    train_dist = np.concatenate([train_dist, valid_dist], axis=0)
    train_y = np.concatenate([train_y, valid_y], axis=0)

It's test time, concatting validation data to train


In [9]:
train_svi_global = np.swapaxes(train_svi_global, 1, 2)
valid_svi_global = np.swapaxes(valid_svi_global, 1, 2)
test_svi_global = np.swapaxes(test_svi_global, 1, 2)

temporal_feature_size = train_svi_global.shape[-1]
no_locations = train_svi_global.shape[2]
static_feature_size = train_static.shape[1]

conf.temporal_feature_size = temporal_feature_size
conf.no_locations = no_locations
conf.static_feature_size = static_feature_size

sess = tf.compat.v1.Session()

castnet = hughes_castnet_model.CASTNet(sess=sess, conf=conf)
print(train_svi_local.shape[0])

2023-09-19 14:41:28.936655: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-09-19 14:41:28.936712: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: s1cmp008.pax.tufts.edu
2023-09-19 14:41:28.936722: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: s1cmp008.pax.tufts.edu
2023-09-19 14:41:28.936901: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.104.5
2023-09-19 14:41:28.936948: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.104.5
2023-09-19 14:41:28.936957: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.104.5


Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/cluster/home/kheuto01/code/opioid-overdose-models/CASTNet/hughes-CASTNet/hughes_castnet_model.py:70: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  self.local_lstm_cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(self.HIDDEN_LAYER_SIZE)
/cluster/home/kheuto01/code/opioid-overdose-models/CASTNet/hughes-CASTNet/hughes_castnet_model.py:89: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  global_lstm_cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(self.HIDDEN_LAYER_SIZE)


12960


In [ ]:
castnet.train(train_svi_local, train_svi_global, train_static, train_sample_indices, train_dist, train_y, valid_svi_local, valid_svi_global, valid_static, valid_sample_indices, valid_dist, valid_y, test_svi_local, test_svi_global, test_static, test_sample_indices, test_dist, test_y)

2023-09-19 14:41:33.127228: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled


Iter: 1.0, total_loss: 3.299, regres_loss: 1.144, gl_loss: 0.158, ortho_loss: 1.997
Iter: 2.0, total_loss: 3.317, regres_loss: 1.192, gl_loss: 0.128, ortho_loss: 1.997
Iter: 3.0, total_loss: 3.254, regres_loss: 1.153, gl_loss: 0.104, ortho_loss: 1.997
Iter: 4.0, total_loss: 3.243, regres_loss: 1.162, gl_loss: 0.083, ortho_loss: 1.997
Iter: 5.0, total_loss: 3.184, regres_loss: 1.121, gl_loss: 0.066, ortho_loss: 1.997
Iter: 6.0, total_loss: 3.232, regres_loss: 1.183, gl_loss: 0.052, ortho_loss: 1.997
Iter: 7.0, total_loss: 3.136, regres_loss: 1.092, gl_loss: 0.046, ortho_loss: 1.997
Iter: 8.0, total_loss: 3.12, regres_loss: 1.081, gl_loss: 0.042, ortho_loss: 1.997
Iter: 9.0, total_loss: 3.135, regres_loss: 1.097, gl_loss: 0.04, ortho_loss: 1.997
Iter: 10.0, total_loss: 3.23, regres_loss: 1.194, gl_loss: 0.038, ortho_loss: 1.997
Epoch: 1, total_loss: 3.203, regres_loss: 1.137, gl_loss: 0.074, ortho_loss: 1.991
-----------------------------------------------------------------------------
I

Iter: 84.0, total_loss: 2.21, regres_loss: 0.604, gl_loss: 0.008, ortho_loss: 1.598
Iter: 85.0, total_loss: 2.669, regres_loss: 0.661, gl_loss: 0.01, ortho_loss: 1.998
Iter: 86.0, total_loss: 2.714, regres_loss: 0.707, gl_loss: 0.01, ortho_loss: 1.998
Iter: 87.0, total_loss: 2.755, regres_loss: 0.747, gl_loss: 0.011, ortho_loss: 1.998
Iter: 88.0, total_loss: 2.774, regres_loss: 0.765, gl_loss: 0.011, ortho_loss: 1.998
Iter: 89.0, total_loss: 2.823, regres_loss: 0.815, gl_loss: 0.011, ortho_loss: 1.998
Iter: 90.0, total_loss: 2.811, regres_loss: 0.802, gl_loss: 0.011, ortho_loss: 1.998
Iter: 91.0, total_loss: 2.94, regres_loss: 0.931, gl_loss: 0.011, ortho_loss: 1.998
Iter: 92.0, total_loss: 2.945, regres_loss: 0.936, gl_loss: 0.012, ortho_loss: 1.998
Iter: 93.0, total_loss: 2.814, regres_loss: 0.805, gl_loss: 0.012, ortho_loss: 1.998
Epoch: 9, total_loss: 2.801, regres_loss: 0.799, gl_loss: 0.011, ortho_loss: 1.991
-----------------------------------------------------------------------

Epoch: 16, total_loss: 2.601, regres_loss: 0.599, gl_loss: 0.011, ortho_loss: 1.991
-----------------------------------------------------------------------------
Iter: 167.0, total_loss: 1.489, regres_loss: 0.284, gl_loss: 0.007, ortho_loss: 1.199
Iter: 168.0, total_loss: 2.512, regres_loss: 0.504, gl_loss: 0.01, ortho_loss: 1.998
Iter: 169.0, total_loss: 2.559, regres_loss: 0.551, gl_loss: 0.011, ortho_loss: 1.998
Iter: 170.0, total_loss: 2.54, regres_loss: 0.532, gl_loss: 0.011, ortho_loss: 1.998
Iter: 171.0, total_loss: 2.528, regres_loss: 0.52, gl_loss: 0.011, ortho_loss: 1.998
Iter: 172.0, total_loss: 2.577, regres_loss: 0.568, gl_loss: 0.011, ortho_loss: 1.998
Iter: 173.0, total_loss: 2.618, regres_loss: 0.609, gl_loss: 0.012, ortho_loss: 1.998
Iter: 174.0, total_loss: 2.554, regres_loss: 0.545, gl_loss: 0.012, ortho_loss: 1.998
Iter: 175.0, total_loss: 2.562, regres_loss: 0.551, gl_loss: 0.013, ortho_loss: 1.998
Iter: 176.0, total_loss: 2.62, regres_loss: 0.609, gl_loss: 0.013, 

Iter: 248.0, total_loss: 2.336, regres_loss: 0.328, gl_loss: 0.01, ortho_loss: 1.998
Iter: 249.0, total_loss: 2.355, regres_loss: 0.347, gl_loss: 0.01, ortho_loss: 1.998
Epoch: 24, total_loss: 2.331, regres_loss: 0.328, gl_loss: 0.011, ortho_loss: 1.991
-----------------------------------------------------------------------------
Iter: 250.0, total_loss: 0.899, regres_loss: 0.096, gl_loss: 0.005, ortho_loss: 0.799
Iter: 251.0, total_loss: 2.293, regres_loss: 0.283, gl_loss: 0.013, ortho_loss: 1.998
Iter: 252.0, total_loss: 2.303, regres_loss: 0.293, gl_loss: 0.012, ortho_loss: 1.998
Iter: 253.0, total_loss: 2.31, regres_loss: 0.3, gl_loss: 0.012, ortho_loss: 1.998
Iter: 254.0, total_loss: 2.302, regres_loss: 0.293, gl_loss: 0.012, ortho_loss: 1.998
Iter: 255.0, total_loss: 2.315, regres_loss: 0.306, gl_loss: 0.012, ortho_loss: 1.998
Iter: 256.0, total_loss: 2.315, regres_loss: 0.307, gl_loss: 0.011, ortho_loss: 1.998
Iter: 257.0, total_loss: 2.341, regres_loss: 0.333, gl_loss: 0.011, o

...

Iter: 1868.0, total_loss: 2.021, regres_loss: 0.022, gl_loss: 0.002, ortho_loss: 1.998
Iter: 1869.0, total_loss: 2.019, regres_loss: 0.019, gl_loss: 0.002, ortho_loss: 1.998
Iter: 1870.0, total_loss: 2.024, regres_loss: 0.024, gl_loss: 0.002, ortho_loss: 1.998
Iter: 1871.0, total_loss: 2.022, regres_loss: 0.023, gl_loss: 0.002, ortho_loss: 1.998
Iter: 1872.0, total_loss: 1.956, regres_loss: 0.021, gl_loss: 0.001, ortho_loss: 1.934
Epoch: 180, total_loss: 2.015, regres_loss: 0.022, gl_loss: 0.002, ortho_loss: 1.991
-----------------------------------------------------------------------------
Iter: 1873.0, total_loss: 2.02, regres_loss: 0.021, gl_loss: 0.002, ortho_loss: 1.998
Iter: 1874.0, total_loss: 2.022, regres_loss: 0.023, gl_loss: 0.002, ortho_loss: 1.998
Iter: 1875.0, total_loss: 2.021, regres_loss: 0.022, gl_loss: 0.002, ortho_loss: 1.998
Iter: 1876.0, total_loss: 2.023, regres_loss: 0.023, gl_loss: 0.002, ortho_loss: 1.998
Iter: 1877.0, total_loss: 2.02, regres_loss: 0.021, gl_

Iter: 1948.0, total_loss: 2.022, regres_loss: 0.023, gl_loss: 0.001, ortho_loss: 1.998
Iter: 1949.0, total_loss: 2.018, regres_loss: 0.019, gl_loss: 0.001, ortho_loss: 1.998
Iter: 1950.0, total_loss: 2.024, regres_loss: 0.025, gl_loss: 0.001, ortho_loss: 1.998
Iter: 1951.0, total_loss: 2.019, regres_loss: 0.02, gl_loss: 0.002, ortho_loss: 1.998
Iter: 1952.0, total_loss: 2.018, regres_loss: 0.019, gl_loss: 0.001, ortho_loss: 1.998
Iter: 1953.0, total_loss: 2.019, regres_loss: 0.02, gl_loss: 0.001, ortho_loss: 1.998
Iter: 1954.0, total_loss: 2.021, regres_loss: 0.022, gl_loss: 0.002, ortho_loss: 1.998
Iter: 1955.0, total_loss: 2.02, regres_loss: 0.021, gl_loss: 0.001, ortho_loss: 1.998
Epoch: 188, total_loss: 2.014, regres_loss: 0.021, gl_loss: 0.001, ortho_loss: 1.991
-----------------------------------------------------------------------------
Iter: 1956.0, total_loss: 1.616, regres_loss: 0.017, gl_loss: 0.001, ortho_loss: 1.598
Iter: 1957.0, total_loss: 2.017, regres_loss: 0.018, gl_l

Iter: 2029.0, total_loss: 2.02, regres_loss: 0.021, gl_loss: 0.002, ortho_loss: 1.998
Iter: 2030.0, total_loss: 2.02, regres_loss: 0.02, gl_loss: 0.002, ortho_loss: 1.998
Iter: 2031.0, total_loss: 2.019, regres_loss: 0.019, gl_loss: 0.002, ortho_loss: 1.998
Iter: 2032.0, total_loss: 2.022, regres_loss: 0.022, gl_loss: 0.002, ortho_loss: 1.998
Iter: 2033.0, total_loss: 2.021, regres_loss: 0.022, gl_loss: 0.002, ortho_loss: 1.998
Iter: 2034.0, total_loss: 2.019, regres_loss: 0.019, gl_loss: 0.002, ortho_loss: 1.998
Iter: 2035.0, total_loss: 2.018, regres_loss: 0.019, gl_loss: 0.002, ortho_loss: 1.998
Iter: 2036.0, total_loss: 2.018, regres_loss: 0.019, gl_loss: 0.001, ortho_loss: 1.998
Iter: 2037.0, total_loss: 2.02, regres_loss: 0.021, gl_loss: 0.001, ortho_loss: 1.998
Iter: 2038.0, total_loss: 2.021, regres_loss: 0.022, gl_loss: 0.002, ortho_loss: 1.998
Epoch: 196, total_loss: 2.014, regres_loss: 0.02, gl_loss: 0.002, ortho_loss: 1.991
--------------------------------------------------

Iter: 2111.0, total_loss: 2.02, regres_loss: 0.021, gl_loss: 0.001, ortho_loss: 1.998
Epoch: 203, total_loss: 2.014, regres_loss: 0.021, gl_loss: 0.001, ortho_loss: 1.991
-----------------------------------------------------------------------------
Iter: 2112.0, total_loss: 1.612, regres_loss: 0.013, gl_loss: 0.001, ortho_loss: 1.598
Iter: 2113.0, total_loss: 2.017, regres_loss: 0.018, gl_loss: 0.001, ortho_loss: 1.998
Iter: 2114.0, total_loss: 2.017, regres_loss: 0.018, gl_loss: 0.001, ortho_loss: 1.998
Iter: 2115.0, total_loss: 2.017, regres_loss: 0.018, gl_loss: 0.001, ortho_loss: 1.998
Iter: 2116.0, total_loss: 2.021, regres_loss: 0.021, gl_loss: 0.002, ortho_loss: 1.998
Iter: 2117.0, total_loss: 2.021, regres_loss: 0.022, gl_loss: 0.002, ortho_loss: 1.998
Iter: 2118.0, total_loss: 2.018, regres_loss: 0.019, gl_loss: 0.001, ortho_loss: 1.998
Iter: 2119.0, total_loss: 2.024, regres_loss: 0.024, gl_loss: 0.002, ortho_loss: 1.998
Iter: 2120.0, total_loss: 2.021, regres_loss: 0.022, gl

Iter: 2191.0, total_loss: 2.022, regres_loss: 0.022, gl_loss: 0.002, ortho_loss: 1.998
Iter: 2192.0, total_loss: 2.019, regres_loss: 0.02, gl_loss: 0.002, ortho_loss: 1.998
Iter: 2193.0, total_loss: 2.022, regres_loss: 0.023, gl_loss: 0.002, ortho_loss: 1.998
Iter: 2194.0, total_loss: 2.018, regres_loss: 0.02, gl_loss: 0.001, ortho_loss: 1.998
Epoch: 211, total_loss: 2.014, regres_loss: 0.021, gl_loss: 0.002, ortho_loss: 1.991
-----------------------------------------------------------------------------
Iter: 2195.0, total_loss: 1.212, regres_loss: 0.013, gl_loss: 0.001, ortho_loss: 1.199
Iter: 2196.0, total_loss: 2.02, regres_loss: 0.022, gl_loss: 0.001, ortho_loss: 1.998
Iter: 2197.0, total_loss: 2.017, regres_loss: 0.018, gl_loss: 0.001, ortho_loss: 1.998
Iter: 2198.0, total_loss: 2.015, regres_loss: 0.016, gl_loss: 0.001, ortho_loss: 1.998
Iter: 2199.0, total_loss: 2.017, regres_loss: 0.018, gl_loss: 0.001, ortho_loss: 1.998
Iter: 2200.0, total_loss: 2.016, regres_loss: 0.017, gl_l

In [ ]:
model_save_path = os.path.join('./', 'Results')
    
if cook_county:
    model_save_path = os.path.join(model_save_path, 'Chicago')
else:
    model_save_path = os.path.join(model_save_path, 'MA')

castnet.save(model_save_path)
castnet.save_results()